# Proyecto Heart Disease Prediction

## Pipeline de Entrenamiento, Optimización y Serialización

**Objetivo**
Transformar los datos limpios (`interim`) en artefactos de Machine Learning listos para producción. Este notebook ejecuta un flujo de trabajo completo que incluye ingeniería de características, imputación, escalado, búsqueda de hiperparámetros (GridSearch) y persistencia de múltiples modelos.

**Metodología:**
1.  **Ingeniería de Features:** Transformación de variables categóricas (One-Hot Encoding).
2.  **Preprocesamiento Robusto:** Creación y guardado de `Imputer` y `Scaler` para garantizar consistencia en inferencia.
3.  **Hyperparameter Tuning:** Optimización de 4 algoritmos clave usando Validación Cruzada (5-Fold CV):
    * *Logistic Regression*
    * *Support Vector Machine (SVM)*
    * *Decision Tree*
    * *Random Forest*
4.  **Persistencia MLOps:** Serialización de todos los modelos y generación de un archivo `metrics.json` para el Dashboard de la App.

---
* **Autor:** [Feliz Florian Jose Luis]
* **Fecha:** [12/12/2025]
* **Input:** `data/02_interim/heart_disease_prediction_cleaned.csv`
* **Output:** Modelos en `data/05_models` y Artefactos en `artefacts/`.
---

In [1]:
# --- Librerías estándar ---
import pandas as pd
import numpy as np
import pickle
import json
import os
from google.colab import drive

# --- Librerías de visualización ---
import matplotlib.pyplot as plt
import seaborn as sns


# --- Scikit-Learn ---
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, classification_report

# Montaje de Unidad (Persistencia en Google Drive)
drive.mount('/content/drive')

# --- DEFINICIÓN DE RUTAS (CONSTANTES) ---

# Ruta Base (Ancla del Proyecto)
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/heart_disease_prediction_mlops"

# Inputs (Datos Limpios del EDA)
INTERIM_DATA_PATH = os.path.join(PROJECT_DIR, "data", "02_interim", "heart_disease_prediction_cleaned.csv")

# Outputs (Destinos de guardado)
MODELS_DIR = os.path.join(PROJECT_DIR, "data", "05_models")
ARTEFACTS_DIR = os.path.join(PROJECT_DIR, "artefacts")
REPORTS_DIR = os.path.join(PROJECT_DIR, "data", "06_reporting")

# --- VALIDACIÓN DE DIRECTORIOS ---

# Creamos directorios si no existen
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(ARTEFACTS_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)
print("\n")
print(f"Entorno Configurado.\n")
print(f"Input Data: {INTERIM_DATA_PATH}\n")
print(f"Directorio de Modelos: {MODELS_DIR}\n")
print(f"Directorio de Artefactos: {ARTEFACTS_DIR}\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Entorno Configurado.

Input Data: /content/drive/MyDrive/Colab Notebooks/heart_disease_prediction_mlops/data/02_interim/heart_disease_prediction_cleaned.csv

Directorio de Modelos: /content/drive/MyDrive/Colab Notebooks/heart_disease_prediction_mlops/data/05_models

Directorio de Artefactos: /content/drive/MyDrive/Colab Notebooks/heart_disease_prediction_mlops/artefacts



## 1. Pipeline de Datos y Generación de Artefactos

Transformación de variables categóricas y normalización numérica. El objetivo es preparar los datos y, simultáneamente, exportar las reglas de transformación para el despliegue.

**Puntos Críticos de MLOps:**
1.  **Validación Futura:** Guardamos `features_names.pkl` para garantizar que, cuando despleguemos la aplicación, podamos rechazar archivos o datos que no tengan las columnas correctas.
2.  **Reproducibilidad:** Exportamos el `Imputer` y el `Scaler`. Así, cuando la App reciba un paciente nuevo en el futuro, aplicará la misma mediana y la misma escala que usamos aquí, garantizando predicciones coherentes.

In [2]:
# Carga de Datos
try:
    print("\n")
    df = pd.read_csv(INTERIM_DATA_PATH)
    print(f"Datos cargados: {df.shape}\n")
except FileNotFoundError:
    print("ERROR CRÍTICO: No se encuentra el archivo interim. Ejecuta el Notebook 01_EDA_Analysis primero.\n")

# One-Hot Encoding (Transformación de Texto a Números)
df_encoded = pd.get_dummies(df, drop_first=True)

# Separación Features (X) y Target (y)
X = df_encoded.drop(columns=['HeartDisease'])
y = df_encoded['HeartDisease']

# --- MLOps: GUARDAR NOMBRES DE FEATURES ---

feature_names = list(X.columns)
with open(os.path.join(ARTEFACTS_DIR, "features_names.pkl"), "wb") as f:
    pickle.dump(feature_names, f)
print(f"Lista de features guardada ({len(feature_names)} variables).\n")

# División Train/Test (Estratificada)
# Mantenemos la proporción de enfermos/sanos igual en ambos sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Imputación (Manejo de Nulos)
# Usamos la Mediana para rellenar los huecos de Cholesterol/RestingBP generados en la limpieza
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# --- MLOps: GUARDAR IMPUTER ---

with open(os.path.join(ARTEFACTS_DIR, "imputer.pkl"), "wb") as f:
    pickle.dump(imputer, f)

# Escalado (Estandarización)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# --- MLOps: GUARDAR SCALER ---

with open(os.path.join(ARTEFACTS_DIR, "scaler.pkl"), "wb") as f:
    pickle.dump(scaler, f)

print("Preprocesamiento completado y artefactos guardados en 'artefacts/'.\n")



Datos cargados: (918, 12)

Lista de features guardada (15 variables).

Preprocesamiento completado y artefactos guardados en 'artefacts/'.



## 2. Entrenamiento y Optimización de Hiperparámetros

Utilizamos **`GridSearchCV`** para encontrar la mejor configuración para cada algoritmo.
* **Estrategia:** Probamos múltiples combinaciones (Solvers, Kernels, Profundidades) usando Validación Cruzada de 5 pliegues (5-Fold CV).
* **Métrica de Selección:** Optimizamos buscando el mejor **Recall**, ya que en medicina es prioritario detectar a todos los enfermos.

In [3]:
# Diccionario para almacenar los mejores modelos encontrados
best_models = {}

print("\n INICIANDO OPTIMIZACIÓN DE HIPERPARÁMETROS (GridSearch)...")

# --- REGRESIÓN LOGÍSTICA ---
print("\n Optimizando Logistic Regression...")
param_grid_lr = {
    'solver': ['liblinear', 'lbfgs'],
    'C': [0.01, 0.1, 1, 10, 100]
}
grid_lr = GridSearchCV(LogisticRegression(random_state=42, max_iter=1000), param_grid_lr, cv=5, scoring='recall')
grid_lr.fit(X_train_scaled, y_train)
best_models["logistic_regression"] = grid_lr.best_estimator_
print(f"   -> Mejores Params: {grid_lr.best_params_}")

# --- SUPPORT VECTOR MACHINE (SVM) ---
print("\n Optimizando SVM...")
param_grid_svm = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto']
}
# 'probability=True' es vital para que la App muestre la confianza de la predicción
grid_svm = GridSearchCV(SVC(probability=True, random_state=42), param_grid_svm, cv=5, scoring='recall')
grid_svm.fit(X_train_scaled, y_train)
best_models["support_vector_machine"] = grid_svm.best_estimator_
print(f"   -> Mejores Params: {grid_svm.best_params_}")

# --- DECISION TREE ---
print("\n Optimizando Decision Tree...")
param_grid_dt = {
    'max_depth': [3, 5, 7, 10, None],
    'class_weight': ['balanced', None],
    'min_samples_split': [2, 5, 10]
}
grid_dt = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid_dt, cv=5, scoring='recall')
grid_dt.fit(X_train_scaled, y_train)
best_models["decision_tree"] = grid_dt.best_estimator_
print(f"   -> Mejores Params: {grid_dt.best_params_}")

# --- RANDOM FOREST ---
print("\n Optimizando Random Forest...")
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, None],
    'class_weight': ['balanced', None]
}
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, scoring='recall')
grid_rf.fit(X_train_scaled, y_train)
best_models["random_forest"] = grid_rf.best_estimator_
print(f"   -> Mejores Params: {grid_rf.best_params_}")


 INICIANDO OPTIMIZACIÓN DE HIPERPARÁMETROS (GridSearch)...

 Optimizando Logistic Regression...
   -> Mejores Params: {'C': 0.01, 'solver': 'lbfgs'}

 Optimizando SVM...
   -> Mejores Params: {'C': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}

 Optimizando Decision Tree...
   -> Mejores Params: {'class_weight': 'balanced', 'max_depth': 3, 'min_samples_split': 2}

 Optimizando Random Forest...
   -> Mejores Params: {'class_weight': None, 'max_depth': 5, 'n_estimators': 200}


## 3. Evaluación Comparativa y Serialización

1.  **Evaluación:** Probamos los modelos optimizados en el **Test Set** (datos nunca vistos). Calculamos *Accuracy*, *Recall* y *F1-Score*.
2.  **Generación de Métricas:** Guardamos los resultados en `metrics.json` para alimentar el Dashboard de Streamlit.
3.  **Serialización:** Guardamos cada modelo individualmente como `.pkl`.

In [4]:
# Diccionario maestro para el JSON de métricas
final_metrics = {}

print("\n EVALUACIÓN FINAL EN TEST SET Y GUARDADO:")
print("=" * 80 +"\n")

for name_key, model in best_models.items():
    # Predicción
    y_pred = model.predict(X_test_scaled)

    # Cálculo de Métricas
    acc = accuracy_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Almacenamiento en Memoria (JSON Struct)
    final_metrics[name_key] = {
        "accuracy": round(acc, 4),
        "recall": round(rec, 4),
        "f1_score": round(f1, 4)
    }

    print(f" {name_key.upper():<20} | Recall: {rec:.2%} | F1-Score: {f1:.2%} | Accuracy: {acc:.2%}")

    # Serialización del Modelo (.pkl)
    # Nomenclatura: model_nombre_del_algoritmo.pkl
    filename = f"model_{name_key}.pkl"
    path = os.path.join(MODELS_DIR, filename)

    with open(path, "wb") as f:
        pickle.dump(model, f)
    print(f"   Modelo guardado en: {filename}\n")

print("=" * 80)

# Guardado del Archivo de Métricas (JSON)
metrics_path = os.path.join(REPORTS_DIR, "metrics.json")
with open(metrics_path, "w") as f:
    json.dump(final_metrics, f, indent=4)

print(f"Archivo 'metrics.json' generado en: {REPORTS_DIR}\n")
print("Pipeline finalizado exitosamente. Listos para Streamlit.\n")


 EVALUACIÓN FINAL EN TEST SET Y GUARDADO:

 LOGISTIC_REGRESSION  | Recall: 89.22% | F1-Score: 89.66% | Accuracy: 88.59%
   Modelo guardado en: model_logistic_regression.pkl

 SUPPORT_VECTOR_MACHINE | Recall: 89.22% | F1-Score: 88.35% | Accuracy: 86.96%
   Modelo guardado en: model_support_vector_machine.pkl

 DECISION_TREE        | Recall: 75.49% | F1-Score: 77.78% | Accuracy: 76.09%
   Modelo guardado en: model_decision_tree.pkl

 RANDOM_FOREST        | Recall: 87.25% | F1-Score: 86.41% | Accuracy: 84.78%
   Modelo guardado en: model_random_forest.pkl

Archivo 'metrics.json' generado en: /content/drive/MyDrive/Colab Notebooks/heart_disease_prediction_mlops/data/06_reporting

Pipeline finalizado exitosamente. Listos para Streamlit.



## 4. Conclusiones y Próximos Pasos

El pipeline de entrenamiento ha finalizado con éxito. Se han logrado los siguientes hitos:

1.  **Modelado:** Se han entrenado y optimizado 4 algoritmos diferentes, utilizando validación cruzada para evitar el sobreajuste.
2.  **Foco Clínico:** La optimización se centró en la métrica **Recall**, priorizando la detección de casos positivos (enfermos).
3.  **Persistencia Completa:** Se han exportado todos los componentes necesarios para construir la aplicación web:
    * `scaler.pkl` y `imputer.pkl` para preprocesar nuevos datos.
    * 4 Modelos `.pkl` (`Logistic Regression`,`Support Vector Machine (SVM)`, `Decision Tree`, `Random Forest`) para predicción flexible.
    * `metrics.json` para la visualización comparativa de rendimiento.

**Siguiente Fase:** Desarrollo de la Interfaz de Usuario con **Streamlit**.